In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 58,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from lxml import etree"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Input format\n",
    "\n",
    "The input consists of two files:\n",
    "\n",
    "* a file with the first sentences in each pair\n",
    "* a file with the second sentences in each pair\n",
    "\n",
    "The sentences are tokenized.\n",
    "\n",
    "Please check STSint.input.*.sent1.txt and STSint.*.input.sent2.txt\n",
    "\n",
    "Participants can also use the input sentences with gold standard chunks:\n",
    "\n",
    "* a file with the first sentences in each pair, with '[' and ']' to mark chunks\n",
    "* a file with the second sentences in each pair, with '[' and ']' to mark chunks\n",
    "\n",
    "Please check STSint.input.*.sent1.chunk.txt and STSint.input.*.sent2.chunk.txt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [],
   "source": [
    "# loading unchunked headlines\n",
    "unchunked_path_1 = \"test_goldstandard/STSint.testinput.headlines.sent1.txt\"\n",
    "unchunked_path_2 = \"test_goldstandard/STSint.testinput.headlines.sent2.txt\"\n",
    "\n",
    "headlines_sentance1 = pd.read_csv(unchunked_path_1, dtype=str, delimiter=\"}\", header=None)\n",
    "headlines_sentance1.columns = [\"headlines_sentance1\"]\n",
    "\n",
    "headlines_sentance2 = pd.read_csv(unchunked_path_2, dtype=str, delimiter=\"}\", header=None)\n",
    "headlines_sentance2.columns = [\"headlines_sentance2\"]\n",
    "\n",
    "headlines = pd.concat([headlines_sentance1, headlines_sentance2], axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "# loading unchunked images\n",
    "unchunked_path_1 = \"test_goldstandard/STSint.testinput.images.sent1.txt\"\n",
    "unchunked_path_2 = \"test_goldstandard/STSint.testinput.images.sent2.txt\"\n",
    "\n",
    "image_sentance1 = pd.read_csv(unchunked_path_1, dtype=str, delimiter=\"}\", header=None)\n",
    "image_sentance1.columns = [\"image_sentance1\"]\n",
    "\n",
    "image_sentance2 = pd.read_csv(unchunked_path_2, dtype=str, delimiter=\"}\", header=None)\n",
    "image_sentance2.columns = [\"image_sentance2\"]\n",
    "\n",
    "images = pd.concat([image_sentance1, image_sentance2], axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "# loading unchunked students\n",
    "unchunked_path_1 = \"test_goldstandard/STSint.testinput.answers-students.sent1.txt\"\n",
    "unchunked_path_2 = \"test_goldstandard/STSint.testinput.answers-students.sent2.txt\"\n",
    "\n",
    "student_sentance1 = pd.read_csv(unchunked_path_1, dtype=str, delimiter=\"}\", header=None)\n",
    "student_sentance1.columns = [\"student_sentance1\"]\n",
    "\n",
    "student_sentance2 = pd.read_csv(unchunked_path_2, dtype=str, delimiter=\"}\", header=None)\n",
    "student_sentance2.columns = [\"student_sentance2\"]\n",
    "\n",
    "students = pd.concat([student_sentance1, student_sentance2], axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "metadata": {},
   "outputs": [],
   "source": [
    "def chunk2list(chunks:str) -> list:\n",
    "    \"\"\"\n",
    "    Takes str that is all chunks from a chunked sentance and returns a list of all the chunks as seperate items \n",
    "    \"\"\"\n",
    "    chunks = chunks.replace('[', '')\n",
    "    chunks = chunks.replace(']', '')\n",
    "    chunks = chunks.replace('   ', '|')\n",
    "    split = chunks.split('|')\n",
    "    return split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "metadata": {},
   "outputs": [],
   "source": [
    "# loading chunked headlines\n",
    "chunked_path_1 = \"test_goldstandard/STSint.testinput.headlines.sent1.chunk.txt\"\n",
    "chunked_path_2 = \"test_goldstandard/STSint.testinput.headlines.sent2.chunk.txt\"\n",
    "\n",
    "headlines_chunked_sentance1 = pd.read_csv(chunked_path_1, dtype=str, delimiter=\"}\", header=None)\n",
    "headlines_chunked_sentance1.columns = [\"headlines_chunked_sentance1\"]\n",
    "\n",
    "headlines_chunked_sentance2 = pd.read_csv(chunked_path_2, dtype=str, delimiter=\"}\", header=None)\n",
    "headlines_chunked_sentance2.columns = [\"headlines_chunked_sentance2\"]\n",
    "\n",
    "headlines_chunked = pd.concat([headlines_chunked_sentance1, headlines_chunked_sentance2], axis=1)\n",
    "\n",
    "# convert chunks from str to list\n",
    "headlines_chunked['headlines_chunked_sentance1'] = headlines_chunked['headlines_chunked_sentance1'].apply(chunk2list)\n",
    "headlines_chunked['headlines_chunked_sentance2'] = headlines_chunked['headlines_chunked_sentance2'].apply(chunk2list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 72,
   "metadata": {},
   "outputs": [],
   "source": [
    "headlines_aligned_path = \"test_goldstandard/STSint.testinput.headlines.wa\" \n",
    "\n",
    "with open(headlines_aligned_path, 'r') as file:\n",
    "    file_content = file.read()\n",
    "\n",
    "# <==> and & break xml loaders so it needs to be replaces with something else\n",
    "modified_content = file_content.replace('<==>', 'ARROWS_PLACEHOLDER').replace('&', 'AMPERSAND_PLACEHOLDER')\n",
    "# it also needs a root wrapped to function properly \n",
    "modified_content = f'<root>{modified_content}</root>'\n",
    "\n",
    "modified_file_path = 'test_goldstandard/STSint.testinput.headlines.fixedarrows.wa'\n",
    "with open(modified_file_path, 'w') as modified_file:\n",
    "    modified_file.write(modified_content)\n",
    "\n",
    "# Parse the modified file using ElementTree\n",
    "tree = etree.parse(modified_file_path)\n",
    "root = tree.getroot()\n",
    "\n",
    "# function for printing XML\n",
    "def prettyprint(element, **kwargs):\n",
    "    xml = etree.tostring(element, pretty_print=True, **kwargs)\n",
    "    print(xml.decode(), end='')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 85,
   "metadata": {},
   "outputs": [],
   "source": [
    "# get ansewrs\n",
    "alignments_data = []\n",
    "\n",
    "for alignment in root.xpath('//alignment'):\n",
    "    # Extract relevant information from the alignment element\n",
    "    data = {\n",
    "        'sentence_id': alignment.xpath('ancestor::sentence/@id')[0],\n",
    "        'alignment_text': alignment.text\n",
    "    }\n",
    "    alignments_data.append(data)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 88,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "6 7 8 ARROWS_PLACEHOLDER 5 6 // EQUI // 5 // for the Philippines ARROWS_PLACEHOLDER to Philippines \n",
      "5 ARROWS_PLACEHOLDER 2 // SIMI // 3 // departs ARROWS_PLACEHOLDER sends \n",
      "9 ARROWS_PLACEHOLDER 0 // NOALI // NIL // Thursday ARROWS_PLACEHOLDER -not aligned- \n",
      "1 ARROWS_PLACEHOLDER 1 // EQUI // 5 // China ARROWS_PLACEHOLDER China \n",
      "2 3 4 ARROWS_PLACEHOLDER 3 4 // REL // 4 // 's Peace Ark ARROWS_PLACEHOLDER aid team \n",
      "\n"
     ]
    }
   ],
   "source": [
    "# test out the format\n",
    "print(alignments_data[0][\"alignment_text\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 107,
   "metadata": {},
   "outputs": [],
   "source": [
    "y = pd.DataFrame(alignments_data)\n",
    "y = y.drop(columns=[\"sentence_id\"])\n",
    "\n",
    "#return to <==> and &\n",
    "def return_characteers(cell: str) -> str:\n",
    "    cell = cell.replace('ARROWS_PLACEHOLDER', '<==>')\n",
    "    cell = cell.replace('AMPERSAND_PLACEHOLDER', '&')\n",
    "    return cell\n",
    "\n",
    "y[\"alignment_text\"] = y[\"alignment_text\"].apply(return_characteers)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 108,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "c:\\Users\\Mati\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\numpy\\core\\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.\n",
      "  return bound(*args, **kwds)\n"
     ]
    }
   ],
   "source": [
    "# generate train test split\n",
    "x = headlines_chunked\n",
    "y = y\n",
    "\n",
    "data = pd.merge(x, y, left_index=True, right_index=True)\n",
    "\n",
    "train, validate, test = np.split(data.sample(frac=1, random_state=42), [int(.6*len(data)), int(.8*len(data))])"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
